In [ ]:
import cv2
import holoviews as hv
from holoviews import opts
import param
import panel as pn
import numpy as np
import holoviews as hv
from holoviews import opts
import numpy as np
import pandas as pd
pn.extension()
hv.extension('bokeh')

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Loading

Use this cell to point to the location of your 3 sources
1. MRI source (this the folder path)
1. Pathology source (this is the image path)
1. Pathology annotated image source (this is the annotated image path)

In [ ]:
mri_folder_path = f'data/Prostate Fused-MRI-Pathology/aaa0051/07-02-2000-PELVISPROSTATE-97855/4.000000-T2 AXIAL SM FOV-36207'
pathology_img_path = 'data/pathology/aaa 0051 BSlice.tif'
pathology_annotated_img_path = 'data/pathology/aaa 0051 BSlice annotated.tif'

In [ ]:
pathology_source = cv2.imread(pathology_img_path)
pathology_annotated_source = cv2.imread(pathology_annotated_img_path)

In [ ]:
import utils

# Select and Upscale the MRI slice

Use the follwing cell to select the slice of the MRI image. Additionally, select the upscaled ascpet ratio of the MRI image.

In [ ]:
mri_viewer = utils.MriVisualizer(mri_path=mri_folder_path)
mri_viewer.panel()

# Downscale the Pathology image

In the following cell, downscale the pathology image. Similarly, there are options to transform the image.

Note: This is normally a very large image file. Downscaling this image as small as possible to find the orientation.
      
**WARNING**: The aspect ratio is of the original image at angle 0. Not the rotated bounds of the chart that are visually displayed.

In [ ]:
pathology_viewer = utils.PathologyVisualizer(pathology_img=pathology_source)
pathology_viewer.panel()

# Landmark Selection

After completing the upscaling and downscaling of both the mri image and pathology image. We will use the next control to select the landmarks.

Use the Point Draw tool to select the points. Refrence to point draw selection can be found here: https://holoviews.org/reference/streams/bokeh/PointDraw.html

Note: You can zoom and pan by using the controls.

**WARNING**: Select the same amount of points in both images. The points are associated by color. A new point will always be colored white upon creation and will be changed to the default color if the context changes.

**WARNING**: If you need to reselect the MRI slice or reorient the pathology image, you can go above and update the values in the above control. However, for it to take effect, you will have rerun the following cell.

In [ ]:
landmarks_viewer = utils.LandmarkSelectorVisualizer(fixed_img=mri_viewer.fixed_image, moving_img=pathology_viewer.moving_image)
landmarks_viewer.panel()

In [ ]:
landmarks_viewer.associated_points_to_DF()

In [ ]:
landmarks_viewer.fixed_pts(in_origin=utils.Origin.BOTTOM_LEFT)

In [ ]:
landmarks_viewer.fixed_pts(in_origin=utils.Origin.TOP_LEFT)

In [ ]:
landmarks_viewer.moving_pts(in_origin=utils.Origin.BOTTOM_LEFT)

In [ ]:
landmarks_viewer.moving_pts(utils.Origin.TOP_LEFT)

In [ ]:
tpsa = utils.ThinPlateSplinesAlgorithm(moving_pts_frame=landmarks_viewer.moving_pts(utils.Origin.TOP_LEFT),
                                       fixed_pts_frame=landmarks_viewer.fixed_pts(utils.Origin.TOP_LEFT))

In [ ]:
def tps():
    moving_warped = tpsa.warp(fixed_img=mri_viewer.fixed_image,moving_img=pathology_viewer.moving_image)
    moving_warped_annotated = tpsa.warp(fixed_img=mri_viewer.fixed_image,moving_img=pathology_viewer.transform_annotation_image(pathology_annotated_source))
    return moving_warped, moving_warped_annotated

# Image Alignment Visualization

The below visualization shows the final transformations of the image registration done in the previous steps. In this visualization, you can see the pathology and mri image overlayed on top of each other. Additionally, you can control the transparency of each layer.

In [ ]:
moving_warped,moving_warped_annotated = tps()
overlay_viewer = utils.OverlayVisualizer(fixed=mri_viewer.fixed_image, 
                                         warped=moving_warped, 
                                         annotated=moving_warped_annotated)
overlay_viewer.panel()

In [ ]:
x = pathology_viewer.transform_annotation_image(pathology_annotated_source)

In [ ]:
(hv.RGB(x) + hv.RGB(pathology_viewer.moving_image) + (hv.RGB(pathology_viewer.moving_image) * hv.RGB(x).opts(alpha=.5))).cols(2)

In [ ]:
np.product(x.shape)